In [2]:
from time import time
start_time = time()

f = open("input.txt", "r")
data = list(map(lambda x: tuple(x.split(" ")), f.read().split("\n")))
arrangements = list(map(lambda x: (x[0], tuple(map(int, x[1].split(",")))), data))

In [3]:
def can_be_made(solution, possibility):
    return all([a == b or b == "?" for (a, b) in zip(solution, possibility)])

dp_map = {}
def gen_combos(n, spaces):
    if spaces == 1:
        return [[n]]
    if (n, spaces) in dp_map:
        return dp_map[(n, spaces)]
    combos = []
    for i in range(n + 1):
        combos += [[i] + new_combo for new_combo in gen_combos(n - i, spaces - 1)]
    dp_map[n, spaces] = combos
    return combos

def generate_group_string(group_sizes, spacers):
    output_string, spacers = spacers[0], spacers[1:]
    for group_size, spacer in zip(group_sizes, spacers):
        output_string += ("#" * group_size) + spacer
    return output_string

def generate_all_combinations(input, groups):
    groups = list(groups)
    num_spacers = len(input) - sum(groups)
    spacing_combinations = list(map(lambda x: list(map(lambda y: "." * y, x)), filter(lambda x: all([v != 0 for v in x[1:-1]]), gen_combos(num_spacers, len(groups) + 1))))
    possibilities = list(filter(lambda x: can_be_made(x, input), map(lambda x: generate_group_string(groups, x), spacing_combinations)))
    return len(possibilities)


print(f"Part 1: {sum([generate_all_combinations(prompt, gaps) for prompt, gaps in arrangements]):,}")

Part 1: 7,195


In [4]:
from functools import lru_cache

print_log = True

@lru_cache(maxsize=1000)
def factorial(n):
    return 1 if n <= 1 else n * factorial(n - 1)

@lru_cache(maxsize=1000)
def ncr(n, r):
    return int(factorial(n) / (factorial(n - r) * factorial(r)))

# Using combinatorics, calculate the number of ways you can arrange a set of groups in a string of just question marks.
def stars_and_bars(n, groups):
    spacers_to_arrange = n - sum(groups) - len(groups) + 1
    cutoffs = len(groups)
    return 0 if spacers_to_arrange < 0 else ncr(spacers_to_arrange + cutoffs, cutoffs)

@lru_cache(maxsize = 10000)
def count_solutions(input, groups):
    groups = list(groups)

    # If it needs to be all periods, check that it can be all periods
    if len(groups) == 0:
        return 1 if all([c != "#" for c in input]) else 0
    
    # Check if the groups are longer than feasible in the size of the current input
    if sum(groups) + len(groups) - 1 > len(input):
        if print_log:
            print((input, groups))
            print(sum(groups) + len(groups) - 1, len(input))
            print("Groups can't fit in input")
        return 0

    # Trim decided periods, as they do not affect the solutions.
    if input[0] == ".":
        if print_log:
            print((input, groups))
            print("Trimming starting period")
        return count_solutions(input[1:], tuple(groups))
    if input[-1] == ".":
        if print_log:
            print((input, groups))
            print("Trimming ending period")
        return count_solutions(input[:-1], tuple(groups))
    
    # Check for all hashes and questions
    if len(groups) == 1 and len(input) == groups[0]:
        return 1 if all([c == "#" or c == "?" for c in input]) else 0
    
    # Check if its all question marks
    if all([c == "?" for c in input]):
        if print_log:
            print((input, groups))
            print("Using Stars and Bars")
        return stars_and_bars(len(input), tuple(groups))
    
    # Check if there are no question marks.
    if all([c != "?" for c in input]):
        if print_log:
            print((input, groups))
            print("Checking if valid solution, no remaining questions")
            print(groups, list(map(len, filter(lambda x: len(x) > 0, input.split(".")))))
        return 1 if groups == list(map(len, filter(lambda x: len(x) > 0, input.split(".")))) else 0
    
    # Check if we can trim the first group from the start.
    if all([c == "#" for c in input[:groups[0]]]):
        return 0 if input[groups[0]] == "#" else count_solutions(input[groups[0] + 1:], tuple(groups[1:]))
    
    # Check if we can trim the ending group.
    if all([c == "#" for c in input[len(input) - groups[-1]:]]):
        if print_log:
            print((input, groups))
            print(f"Trimming end -> {input} -> {input[:len(input) - groups[-1] - 1]}")
        return 0 if input[len(input) - groups[-1] - 1] == "#" else count_solutions(input[:len(input) - groups[-1] - 1], tuple(groups[:-1]))
    
    # Check for early zero returns if there are no feasible # followed by . in the first group[0] chars.
    found_hash = False
    for char in input[:groups[0]]:
        if char == "#":
            found_hash = True
        if char == "." and found_hash:
            return 0
        
    found_hash = False
    for char in reversed(input[len(input) - groups[-1]:]):
        if char == "#":
            found_hash = True
        if char == "." and found_hash:
            return 0
    
    # Check if we need to assume the start or end is a group by necessity.
    if input[0] == "#":
        if print_log:
            print((input, groups))
            print(f"Trimming start by necessity -> ({input} -> {input[groups[0] + 1:]}), {groups} -> {groups[1:]}")
        return count_solutions(input[groups[0] + 1:], tuple(groups[1:])) if all([c == "#" or c == "?" for c in input[:groups[0]]]) and (input[groups[0]] in [".", "?"]) else 0
    
    if input[-1] == "#":
        return count_solutions(input[:len(input) - groups[-1] - 1], tuple(groups[:-1])) if all([c == "#" or c == "?" for c in input[len(input) - groups[-1]:]]) and (input[len(input) - groups[-1] - 1] in [".", "?"]) else 0
    
    # Final Case: Sum both possibilities
    question_index = input.index("?")
    option_a, option_b = input[:question_index] + "." + input[question_index + 1:], input[:question_index] + "#" + input[question_index + 1:]
    if print_log:
        print((input, groups))
        print(f"Splitting options -> \"{input}\" -> \"{option_a}\", \"{option_b}\"")
    return count_solutions(option_a, tuple(groups)) + count_solutions(option_b, tuple(groups))
    

In [5]:
def mult_arrangements(arr, mult):
    return [(input + ("?" + input) * (mult - 1), tuple(list(groups) * mult)) for input, groups in arrangements]

def sum_with_progress(func, arr):
    total_sum = 0
    for i, (input, groups) in enumerate(arr):
        print(f"\r{i} / {len(arr)}, Input: \"{input}\"", end = "")
        total_sum += func(input, groups)
    print("\n")
    return total_sum

print(mult_arrangements(arrangements, 5))

In [ ]:

sum_with_progress(count_solutions, mult_arrangements(arrangements, 5))